In [10]:
import re
import json

with open("row.txt", 'r', encoding="utf-8") as file:
    text = file.read()

receipt_data = {
    "Филиал": None, 
    "БИН": None, 
    "НДС Серия": None, 
    "№": None, 
    "Касса": None, 
    "Смена": None,
    "Порядковый номер чека": None, 
    "Чек": None, 
    "Кассир": None, 
    "Продукты": [], 
    "Банковская карта": None,
    "ИТОГО": None, 
    "Фискальный признак": None, 
    "Время": None, 
    "Местоположение": None,
    "Оператор фискальных данных": None, 
    "ФИСКАЛЬНЫЙ ЧЕК": None,
}

product_pattern = re.compile(
    r"(\d+)\.\s*([^\n]+)\n([\d\s]+,\d+)\s*x\s*([\d\s]+,\d+)\n([\d\s]+,\d+)\nСтоимость\s*\n([\d\s]+,\d+)", re.DOTALL
)

for match in product_pattern.findall(text):
    receipt_data["Продукты"].append({
        "Номер": match[0], 
        "Название": match[1],
        "Количество": match[2].replace(" ", ""), 
        "Цена": match[3].replace(" ", ""),
        "Стоимость": match[4].replace(" ", ""),
    })

patterns = {
    "Филиал": re.compile(r"Филиал\s*(.*?)\n"), 
    "БИН": re.compile(r"БИН\s*(\d+)"),
    "НДС Серия": re.compile(r"НДС Серия\s*(\d+)"), 
    "№": re.compile(r"№\s*(\d+)"),
    "Касса": re.compile(r"Касса\s*(\d+-\d+)"), 
    "Смена": re.compile(r"Смена\s*(\d+)"),
    "Порядковый номер чека": re.compile(r"Порядковый номер чека\s*№(\d+)"),
    "Чек": re.compile(r"Чек\s*№(\d+)"), 
    "Кассир": re.compile(r"Кассир\s*(.*?)"),
    "Банковская карта": re.compile(r"Банковская карта:\s*([\d\s]+,\d+)"),
    "Фискальный признак": re.compile(r"Фискальный признак:\s*(\d+)"),
    "ИТОГО": re.compile(r"ИТОГО:\s*([\d\s]+,\d+)"),
    "Время": re.compile(r"Время:\s*(\d{2}\.\d{2}\.\d{4})\s+(\d{2}:\d{2}:\d{2})"),
    "Местоположение": re.compile(r'(г\.\s*([^,]+),\s*([^,]+),\s*([^,]+),\s*(\d+),\s*(.*))'),
    "Оператор фискальных данных": re.compile(r'(АО"Казахтелеком".*?consumer\.oofd\.kz)'),
    "ИНК ОФД": re.compile(r"ИНК ОФД:\s*(\d+)"),
    "Код ККМ КГД (РНМ)": re.compile(r"Код ККМ КГД \(РНМ\):\s*(\d+)"),
    "ЗНМ": re.compile(r"ЗНМ:\s*([A-Z0-9]+)", re.DOTALL),
}

for key, pattern in patterns.items():
    match = pattern.search(text)
    if match:
        if key == "Время":
            receipt_data[key] = f"{match.group(1)} {match.group(2)}"
        elif key == "Местоположение":
            receipt_data[key] = f"{match.group(1)}, {match.group(2)}"
        else:
            receipt_data[key] = match.group(1).strip()  

print("=" * 130)
print(f"{'ЧЕК':^120}")
print("=" * 130)
if receipt_data["Филиал"]:
    print(f"{receipt_data['Филиал']:^120}")
print("=" * 130)

for key in ["БИН", "НДС Серия", "№", "Касса", "Смена", "Порядковый номер чека", "Чек", "Кассир"]:
    if receipt_data[key]:
        print(f"{key:<25} {receipt_data[key]}")

print("=" * 130)
print("\nПРОДАЖА:")
print("-" * 130)

for product in receipt_data["Продукты"]:
    print(f'\nНомер: {product["Номер"]}')
    print(f'Название: {product["Название"]}')
    print(f'Количество: {product["Количество"]}')
    print(f'Цена: {product["Цена"]}')
    print(f'Стоимость: {product["Стоимость"]}\n')

print("-" * 130)
print(f"{'Банковская карта:':<25} {receipt_data['Банковская карта']}")
print(f"{'ИТОГО:':<25} {receipt_data['ИТОГО']}")
print("-" * 130)
print(f"{'Время:':<25} {receipt_data['Время']}")
print(f"{'Местоположение:':<25} {receipt_data['Местоположение']}")
print("-" * 130)
print(f"{'Фискальный признак:':<25} {receipt_data['Фискальный признак']}")
print(f"{'Оператор фискальных данных:':<25} {receipt_data['Оператор фискальных данных']}")
print("-" * 130)
print(f'ФИСКАЛНЫЙ ЧЕК\nФП')
print(f'ИНК ОФД: {receipt_data["ИНК ОФД"]}')
print(f'Код ККМ КГД (РНМ): {receipt_data["Код ККМ КГД (РНМ)"]}')
print(f'ЗНМ: {receipt_data["ЗНМ"]}')
print("WEBKASSA.KZ")


                                                          ЧЕК                                                           
                                                 ТОО EUROPHARMA Астана                                                  
БИН                       080841000761
НДС Серия                 58001
№                         0014377
Касса                     300-189
Смена                     10
Порядковый номер чека     64
Чек                       2331180266

ПРОДАЖА:
----------------------------------------------------------------------------------------------------------------------------------

Номер: 1
Название: Натрия хлорид 0,9%, 200 мл, фл
Количество: 2,000
Цена: 154,00
Стоимость: 308,00


Номер: 2
Название: Борный спирт 3%, 20 мл, фл.
Количество: 1,000
Цена: 51,00
Стоимость: 51,00


Номер: 3
Название: Шприц 2 мл, 3-х комп. (Bioject)
Количество: 2,000
Цена: 16,00
Стоимость: 32,00


Номер: 4
Название: Система для инфузии Vogt Medical
Количество: 2,000
Цена: 60,00
Сто